In [3]:
from stack import *
from constants import *

In [4]:
print("Reading folder: {}".format(folder))

Reading folder: ./predict_save/


In [5]:
models = load_models()

Loading models...
11 model families loaded:
 baseline: global_mean, global_median, user_mean, user_median, movie_mean, movie_median, movie_mean_user_std, movie_median_user_std, movie_mean_user_habit_std, movie_median_user_habit_std, movie_mean_user_habit, movie_median_user_habit, ; 
surprise: surprise_svd, surprise_knn, surprise_svd_pp, ; 
surprise_algo_user_std: surprise_svd_user_std, surprise_knn_user_std, ; 
spotlight: spotlight, ; 
spotlight_user_std: spotlight_user_std, ; 
als: als, ; 
als_user_std: als_user_std, ; 
pyfm: pyfm, ; 
pyfm_user_std: pyfm_user_std, ; 
mfrr: mfrr, ; 
mfrr_user_std: mfrr_user_std, ; 



In [6]:
ground_truth = pd.read_csv(folder + "ground_truth.csv", index_col=0)

In [7]:
ground_truth.head()

,User,Movie,Rating
0,1207,877,3
1,2337,179,3
2,4221,971,4
3,2306,148,5
4,3092,188,4


In [8]:
res, predictions = optimize(models, ground_truth, folder)

Loading predictions from ./predict_save/....
[load_predictions] files: ['als_predictions_1:45:59.818126.csv', 'global_mean_predictions.csv', 'global_median_predictions.csv', 'mfrr_predictions_1:31:45.786389.csv', 'movie_mean_predictions.csv', 'movie_mean_user_habit_predictions.csv', 'movie_mean_user_habit_std_predictions.csv', 'movie_mean_user_std_predictions.csv', 'movie_median_predictions.csv', 'movie_median_user_habit_predictions.csv', 'movie_median_user_habit_std_predictions.csv', 'movie_median_user_std_predictions.csv', 'pyfm_predictions.csv', 'spotlight_predictions.csv', 'spotlight_user_std_predictions_3:39:03.098104.csv', 'surprise_knn_predictions.csv', 'surprise_knn_user_std_predictions_0:20:00.919416.csv', 'surprise_svd_pp_predictions.csv', 'surprise_svd_predictions.csv', 'surprise_svd_user_std_predictions_0:12:35.930335.csv', 'user_mean_predictions.csv', 'user_median_predictions.csv']
Reading 1/22 : als_predictions_1:45:59.818126.csv...
Reading 2/22 : global_mean_predictions.

['als',
 'global_mean',
 'global_median',
 'mfrr',
 'movie_mean',
 'movie_mean_user_habit',
 'movie_mean_user_habit_std',
 'movie_mean_user_std',
 'movie_median',
 'movie_median_user_habit',
 'movie_median_user_habit_std',
 'movie_median_user_std',
 'pyfm',
 'spotlight',
 'spotlight_user_std',
 'surprise_knn',
 'surprise_knn_user_std',
 'surprise_svd_pp',
 'surprise_svd',
 'surprise_svd_user_std',
 'user_mean',
 'user_median']

In [18]:
def get_best_weights(res, predictions, ground_truth):
    # Create best dictionnary
    existing_models = predictions.columns[2:].tolist()
    best_dict = {}
    idx = 0
    for model in existing_models:
        best_dict[model] = res.x[idx]
        idx = idx + 1
    
    print("Best weights: \n {}".format(best_dict))
    # test
    rmse = evaluate_stacking(res.x, models, predictions, ground_truth)
    print("Best weights rmse: {}".format(rmse))
    return best_dict, rmse

In [30]:
def predict(weight_dict, models):
    print("Loading predictions....")
    predictions = load_predictions(folder_predict)
    print("Finished loading.")
    
    existing_models = predictions.columns[2:].tolist()
    
    stacked = np.zeros(predictions.shape[0])
    for key, model_fam in models.items():
        for name in model_fam.keys():
            if name in existing_models:
                weight = weight_dict[name]
                print("Stacking {} * {}...".format(weight, name))
                stacked = stacked + weight * predictions[name]
    
    pred = predictions[['User', 'Movie']].copy()
    pred['Rating'] = stacked
    return pred

In [29]:
best_dict, rmse = get_best_weights(res, predictions, ground_truth)

Best weights: 
 {'als': 0.04156705547758971, 'global_mean': 0.04141987786482112, 'global_median': -0.049992873632281526, 'mfrr': 0.0026337763948453188, 'movie_mean': 0.036019683272407996, 'movie_mean_user_habit': 0.054753807715713865, 'movie_mean_user_habit_std': 0.2211152209790129, 'movie_mean_user_std': 0.07042404288603338, 'movie_median': -0.19430758778328971, 'movie_median_user_habit': -0.11959403936217423, 'movie_median_user_habit_std': -0.05553875753153203, 'movie_median_user_std': -0.03680447560310902, 'pyfm': 0.008765438135045127, 'spotlight': 0.01881009729092297, 'spotlight_user_std': 0.006383133180331191, 'surprise_knn': -1.0344699395226705e-05, 'surprise_knn_user_std': 0.021320357335072045, 'surprise_svd_pp': 0.022800519613375873, 'surprise_svd': 0.38529971058007395, 'surprise_svd_user_std': 0.21116440273761694, 'user_mean': 0.17468717610442938, 'user_median': 0.15789733232777103}
Best weights rmse: 0.9893887468678488


In [31]:
pred = predict(best_dict, models)

Loading predictions....
[load_predictions] files: ['als_predictions.csv', 'global_mean_predictions.csv', 'global_median_predictions.csv', 'mfrr_predictions.csv', 'movie_mean_predictions.csv', 'movie_mean_user_habit_predictions.csv', 'movie_mean_user_habit_std_predictions.csv', 'movie_mean_user_std_predictions.csv', 'movie_median_predictions.csv', 'movie_median_user_habit_predictions.csv', 'movie_median_user_habit_std_predictions.csv', 'movie_median_user_std_predictions.csv', 'pyfm_predictions.csv', 'spotlight_predictions.csv', 'spotlight_user_std_predictions.csv', 'surprise_knn_predictions.csv', 'surprise_knn_user_std_predictions.csv', 'surprise_svd_pp_predictions.csv', 'surprise_svd_predictions.csv', 'surprise_svd_user_std_predictions.csv', 'user_mean_predictions.csv', 'user_median_predictions.csv']
Reading 1/22 : als_predictions.csv...
Reading 2/22 : global_mean_predictions.csv...
Reading 3/22 : global_median_predictions.csv...
Reading 4/22 : mfrr_predictions.csv...
Reading 5/22 : mo

In [32]:
pred.head()

,User,Movie,Rating
0,1,4,3.444935
1,1,8,3.997530
2,1,21,3.065812
3,1,102,3.632541
4,1,127,3.264525


In [33]:
pred.shape

(1176952, 3)

In [34]:
submission = create_csv_submission(pred)
submission.to_csv("submission.csv")

Creating submission file...


In [35]:
test_df = load_dataset(test_dataset)

[load_dataset] Valid: (1176952, 3)


In [38]:
test_df.sort_values(by=['User', 'Movie']).head()

,User,Movie,Rating
1713,1,4,3
11826,1,8,3
30302,1,21,3
122581,1,102,3
147207,1,127,3


In [37]:
test_df.shape

(1176952, 3)